# ECG-ViEW - Final 

In [1]:
import pandas as pd
import numpy as np
import pylab as plt
import tensorflow as tf
from tensorflow.python.framework import ops
print(tf.__version__)

#Visualization Libraries
import seaborn as sns

# Size of matplotlib histogram bins
bin_size = 20

2.1.0


In [2]:
df = pd.read_csv('Final_Dataset.csv')
df.head()

,RR,PR,QRS,QT,QTc,P_wave_axis,QRS_axis,T_wave_axis,ACCI,sex,Birthyeargroup,MI
0,769,204,116,364,414,56,-81,43,8,1,12,0
1,1000,220,124,428,428,59,-74,33,8,1,12,0
2,769,198,116,398,453,51,268,36,8,1,12,0
3,1000,220,124,416,416,51,-58,23,8,1,12,0
4,968,228,122,436,442,54,-59,34,9,1,12,0


In [3]:
df.shape

(98903, 12)

In [4]:
df['MI'].value_counts()

0    97863
1     1040
Name: MI, dtype: int64

## Split train and test data

In [5]:
train_data = df.values
train_data.shape

(98903, 12)

In [6]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

# Training data features
train_features = train_data[:, 0:11]

# 'MI' column values
train_target = train_data[:, 11]

# defining the scaler
robust_scaler = RobustScaler().fit(train_data)

# scaling test data using minmax scaling
train_data_robust = robust_scaler.transform(train_data)

# training data features
train_features_robust= train_data_robust[:, 0:11]

# 'MI' column values
train_target_robust = train_data_robust[:, 11]

# Split 80-20 train vs test data
train_x, test_x, train_y, test_y = train_test_split(train_features, 
                                                    train_target, 
                                                    test_size=0.20, 
                                                    random_state=0,
                                                    shuffle=True)

print (train_features.shape, train_target.shape)
print (train_x.shape, train_y.shape)
print (test_x.shape, test_y.shape)

(98903, 11) (98903,)
(79122, 11) (79122,)
(19781, 11) (19781,)


In [7]:
unique, counts = np.unique(train_y, return_counts=True)
dict(zip(unique, counts))

{0: 78282, 1: 840}

In [8]:
unique, counts = np.unique(test_y, return_counts=True)
dict(zip(unique, counts))

{0: 19581, 1: 200}

In [9]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
# minority oversampling using SMOTE
over = SMOTE(sampling_strategy=0.25) # get 25% of the data from minority
under = RandomUnderSampler(sampling_strategy=0.5) # make majority only 50% more than the minority in final dataset
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
x_re, y_re = pipeline.fit_resample(train_features_robust, train_target_robust)

Using TensorFlow backend.


In [10]:
unique, counts = np.unique(y_re, return_counts=True)
dict(zip(unique, counts))

{0.0: 48930, 1.0: 24465}

In [11]:
# Split 80-20 train vs test data
train_x, test_x, train_y, test_y = train_test_split(x_re, 
                                                    y_re, 
                                                    test_size=0.20, 
                                                    random_state=0,
                                                    shuffle=True)

print (train_features.shape, train_target.shape)
print (train_x.shape, train_y.shape)
print (test_x.shape, test_y.shape)

(98903, 11) (98903,)
(58716, 11) (58716,)
(14679, 11) (14679,)


In [12]:
# Check the new distribution in the training and testing sets
unique, counts = np.unique(train_y, return_counts=True)
dict(zip(unique, counts))

{0.0: 39114, 1.0: 19602}

In [13]:
unique, counts = np.unique(test_y, return_counts=True)
dict(zip(unique, counts))

{0.0: 9816, 1.0: 4863}

# Recurrent Neural Network

In [ ]:
# importing libraries
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Layer, GRU, LSTM, Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.layers import LeakyReLU
from keras import regularizers, backend, initializers
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn import preprocessing
from keras.initializers import Ones, Zeros
import keras.backend as K
import time
import gc

class LayerNormalization(Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape

layer_size1 = 12
layer_size2 = 10
layer_size3 = 7
layer_size4 = 5
layer_size5 = 4
layer_size6 = 3
timesteps = 1 # static data
data_dim = 11

X_train = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
X_test = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))
train_y = to_categorical(train_y)

#  use_bias=True, bias_initializer=initializers.Constant(-1)

model = Sequential()
model.add(GRU(layer_size1, return_sequences=True, input_shape=(timesteps, data_dim)))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size2, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size3, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size4, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size5, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size6, return_sequences=False))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Dense(2, activation='softmax'))


start = time.time()
opt = optimizers.Adam(0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# Stop the training when there is no improvement in the validation accuracy for ten consecutive epochs.
early = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0)

# reduces learning rate when a metric has stopped improving
redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=7, verbose=0)

# defining the callbacks list to include the above parameters
callbacks_list = [early, redonplat]

# train the model
history = model.fit(X_train, train_y, epochs=1000, verbose=0, callbacks=callbacks_list, validation_split=0.1)

In [ ]:
# test the model
pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

# get f1 score of the model & print it. The f1 score considers the precision & recall.
f1 = f1_score(test_y, pred_test, average="macro")
print("Test f1 score : %s "% f1)

# get ROC AUC score of the model & print it
roc = roc_auc_score(test_y, pred_test)
print("Test ROC AUC Score : %s "% roc)

# get the accuracy and print it
acc = accuracy_score(test_y, pred_test)
print("Test accuracy score : %s "% acc)

# Save the model
model.save("model.h5")

In [ ]:
# Plot model accuracy evolution
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Plot model loss evolution
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Gaussian Naive Bayes

In [ ]:
# Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

# Takes one class only, but since they contain duplicate information, remove one class
train_y_nb = train_y[:,1]
train_y_nb.reshape(-1, 1)
test_y_nb = test_y[:,]
test_y_nb.reshape(-1, 1)

# Create a Gaussian Classifier
gnb = GaussianNB()

# Train the model using the training sets
gnb.fit(train_x, train_y_nb)

# Predict the response for test dataset
y_pred_gnb = gnb.predict(test_x)

In [ ]:
# Model Accuracy
print("Accuracy:", accuracy_score(test_y_nb, y_pred_gnb))

# Complement Naive Bayes

In [ ]:
# Import Complement Naive Bayes model
from sklearn.naive_bayes import ComplementNB

# Algorithm cannot take negative feature values so min/max them to [0,1]
scaler1 = preprocessing.MinMaxScaler()

scaler1.fit(train_x)

train_x_cnb = scaler1.transform(train_x)

scaler2 = preprocessing.MinMaxScaler()

scaler2.fit(train_x)

test_x_cnb = scaler2.transform(test_x)

print(train_x_cnb.shape)

# Create the classifier
clf = ComplementNB()

# Train the model using the training sets
clf.fit(train_x_cnb, train_y_nb)

# Predict the response for test dataset
y_pred_cnb = clf.predict(test_x_cnb)

In [ ]:
# Model Accuracy
print("Accuracy:", accuracy_score(test_y, y_pred_cnb))